In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv("NewSummary.csv")
df.head()

,Date,Part,Description,Branch,StartCount,EndCount,Difference,Variance,Planner
0,3/26/2025,082458P62T,22 1/8 X 6 1/4- 5/8 P62 TEMP,West Fargo,9.0,0.0,-9.0,-1.00,01LTEA
1,3/26/2025,1502S,3 3/4IN STNRY CONE,Eden Prarie,1772.0,572.0,-1200.0,-0.68,02SPRA
2,3/26/2025,40660046LNC,406X6X46 LHW NO CONES,SpringField,25.0,11.0,-14.0,-0.56,03SPRA
3,3/26/2025,SPTAGW,SPRING TAGS WIRED,SpringField,7231.0,3650.0,-3581.0,-0.50,03SPRA
4,3/26/2025,36260030LNC,362X6X30 LHW NO CONES,SpringField,30.0,24.0,-6.0,-0.20,03SPRA


In [4]:
locations = df['Branch'].unique()
print(f"Locations found: {locations.tolist()}")

Locations found: ['West Fargo', 'Eden Prarie', 'SpringField']


In [5]:
part_counts_by_location = df.groupby('Part')['Branch'].nunique()
parts_in_all_locations = part_counts_by_location[part_counts_by_location == len(locations)].index.tolist()

In [6]:

part_counts_by_location = df.groupby('Part')['Branch'].nunique()
parts_in_all_locations = part_counts_by_location[part_counts_by_location == len(locations)].index.tolist()

print(f"\nFound {len(parts_in_all_locations)} parts present in all locations.") # Corrected count after running code

# Filter the original dataframe for these parts
df_all_locations = df[df['Part'].isin(parts_in_all_locations)]


Found 422 parts present in all locations.


In [7]:
print(f"\nFound {len(parts_in_all_locations)} parts present in all locations.") # Corrected count after running code

# Filter the original dataframe for these parts
df_all_locations = df[df['Part'].isin(parts_in_all_locations)].copy()

# Handle duplicates: Keep the last recorded entry for each Part/Branch combo
# Sort by Date first (if dates varied significantly, this would be important)
# Assuming dates are close enough, just dropping duplicates based on Part/Branch is okay here.
df_all_locations_last = df_all_locations.drop_duplicates(subset=['Part', 'Branch'], keep='last')


Found 422 parts present in all locations.


In [8]:
part_variance_matrix = df_all_locations_last.pivot(index='Part', columns='Branch', values='Variance')

print("\n1. Part Variance Matrix (Parts counted in all 3 locations):")
print(part_variance_matrix)
# Save to CSV
part_variance_matrix.to_csv("PartVarianceMatrix.csv")
print("\nSaved PartVarianceMatrix.csv")


1. Part Variance Matrix (Parts counted in all 3 locations):
Branch           Eden Prarie  SpringField  West Fargo
Part                                                 
1001002BAG            -30.00        -1.00       -1.73
1001004BAG              0.00         4.83       -0.54
1004005BAG             -0.77        -1.00        0.01
1004020BAG             -0.02        24.03        3.40
1004A005BAG            -7.93        -1.00        0.00
...                      ...          ...         ...
WS12X82TGWB115          0.08         0.01        0.00
WS16X82TGWB115          0.01         0.06       -0.01
WS16X8RTGWB115         -0.55        -1.00       -0.32
WS16X8TSWB115           0.05         0.48       -0.15
WS18X8TSWB115           0.00        -0.04        0.00

[422 rows x 3 columns]

Saved PartVarianceMatrix.csv


In [9]:
recounts_by_location = df.groupby('Branch').size()

print("\n2. Total Records (Recounts) by Location:")
print(recounts_by_location)

# Create a DataFrame for saving
location_summary = pd.DataFrame({
    'Location': recounts_by_location.index,
    'TotalRecords': recounts_by_location.values
})
# Save to CSV
location_summary.to_csv("LocationSummary_TotalRecords.csv", index=False)
print("\nSaved LocationSummary_TotalRecords.csv")



2. Total Records (Recounts) by Location:
Branch
Eden Prarie    1048
SpringField    1200
West Fargo     1312
dtype: int64

Saved LocationSummary_TotalRecords.csv


In [13]:
print("\n3. Analysis and Summary Review:")

# Ensure column names are stripped of leading/trailing spaces
df.columns = df.columns.str.strip()

# Overall Statistics
print("\nOverall Statistics:")
print(f"Total Records: {len(df)}")
print(f"Date Range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of Unique Parts: {df['Part'].nunique()}")
print(f"Number of Planners: {df['Planner'].nunique()}")

# Variance Analysis
print("\nVariance Statistics (Overall):")
# Convert 'Variance' column to numeric by removing '%' and converting to float
#df['Variance'] = df['Variance'].str.rstrip('%').astype(float)

# Use the provided 'Variance' column, handling potential non-finite values if any snuck in
valid_variance = df['Variance'].replace([np.inf, -np.inf], np.nan).dropna()
print(valid_variance.describe())
print(f"Median Variance: {valid_variance.median():.2f}%") # Median often better for skewed data

# Variance by Location
print("\nVariance Statistics by Location:")
variance_by_location = df.groupby('Branch')['Variance'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
print(variance_by_location)

# Variance by Planner
print("\nVariance Statistics by Planner:")
variance_by_planner = df.groupby('Planner')['Variance'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
print(variance_by_planner.sort_values(by='median')) # Sort to see potential patterns

# Significant Variances (Absolute value > 50%)
threshold = 50
significant_variances = df[df['Variance'].abs() > threshold].copy()
significant_variances['AbsVariance'] = significant_variances['Variance'].abs()
significant_variances_sorted = significant_variances.sort_values(by='AbsVariance', ascending=False)

print(f"\nNumber of records with absolute variance > {threshold}%: {len(significant_variances)}")
print(f"Percentage of records with absolute variance > {threshold}%: {(len(significant_variances) / len(df) * 100):.2f}%")

print("\nTop 10 Largest Absolute Variances:")
print(significant_variances_sorted[['Date', 'Part', 'Branch', 'StartCount', 'EndCount', 'Difference', 'Variance', 'Planner']].head(10))

# Data Quality Notes
negative_start = df[df['StartCount'] < 0]
print(f"\nNumber of records with negative StartCount: {len(negative_start)}")
if len(negative_start) > 0:
    print("Examples of negative StartCount:")
    print(negative_start[['Date', 'Part', 'Branch', 'StartCount']].head())

scientific_parts = df[df['Part'].str.contains(r'E\+', na=False)]
print(f"\nNumber of records with potential scientific notation Part numbers: {len(scientific_parts)}")
if len(scientific_parts) > 0:
     print("Examples of potential scientific notation Part numbers:")
     print(scientific_parts[['Date', 'Part', 'Description', 'Branch']].head())


3. Analysis and Summary Review:

Overall Statistics:
Total Records: 3560
Date Range: 3/26/2025 to 4/2/2025
Number of Unique Parts: 2003
Number of Planners: 22

Variance Statistics (Overall):
count     3560.000000
mean        49.340185
std       2132.440468
min     -21838.000000
25%         -0.190000
50%         -0.010000
75%          0.060000
max      99999.730000
Name: Variance, dtype: float64
Median Variance: -0.01%

Variance Statistics by Location:
                  mean  median          std       min       max  count
Branch                                                                
Eden Prarie  74.890773  -0.025  3162.468377 -21838.00  99999.73   1048
SpringField  -0.003308  -0.020     3.417024    -33.14     85.59   1200
West Fargo   74.062119   0.000  2086.636930   -121.39  71427.57   1312

Variance Statistics by Planner:
                mean  median          std       min       max  count
Planner                                                             
STOCK      -1.000